---
---
---
## Tarea 4 Problema de misioneros y caníbales 

---


Ali Villegas

Julián Herrera

Alfredo Quintero

In [19]:
class Estado:

    def __init__(self,c,m,b=1):
        self.c = c
        self.m = m
        self.b = b

    def isValid(self):
        return (self.c,self.m) not in [(3,1), (3,2), (2,1), (1,2), (0,1), (0,2)]

    def __repr__(self):
        return "(missionaries: {}, cannibals:{}) b = {}".format(self.c,self.m,self.b)
                                                      
    def __eq__(self, other):
        return self.c == other.c and self.m == other.m and self.b == other.b
    
    def __hash__(self):
        return hash((self.c,self.m,self.b))

failState = Estado(-1,-1)

In [20]:
def memoize(fn, slot=None, maxsize=32):
    if slot:
        def memoized_fn(obj, *args):
            if hasattr(obj, slot):
                return getattr(obj, slot)
            else:
                val = fn(obj, *args)
                setattr(obj, slot, val)
                return val
    else:
        @functools.lru_cache(maxsize=maxsize)
        def memoized_fn(*args):
            return fn(*args)

    return memoized_fn

class PriorityQueue:

    def __init__(self, order='min', f=lambda x: x):
        self.heap = []
        if order == 'min':
            self.f = f
        elif order == 'max':  # now item with max f(x)
            self.f = lambda x: -f(x)  # will be popped first
        else:
            raise ValueError("Order must be either 'min' or 'max'.")

    def append(self, item):
        heapq.heappush(self.heap, (self.f(item), item))

    def extend(self, items):
        for item in items:
            self.append(item)

    def pop(self):
        if self.heap:
            return heapq.heappop(self.heap)[1]
        else:
            raise Exception('Trying to pop from empty PriorityQueue.')

    def __len__(self):
        return len(self.heap)

    def __contains__(self, key):
        return any([item == key for _, item in self.heap])

    def __getitem__(self, key):
        for value, item in self.heap:
            if item == key:
                return value
        raise KeyError(str(key) + " is not in the priority queue")

    def __delitem__(self, key):
        try:
            del self.heap[[item == key for _, item in self.heap].index(True)]
        except ValueError:
            raise KeyError(str(key) + " is not in the priority queue")
        heapq.heapify(self.heap)

In [21]:
class Problem(object):
    def __init__(self, initial, goal=None):
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        raise NotImplementedError

    def result(self, state, action):
        raise NotImplementedError

    def goal_test(self, state):
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        return c + 1

    def value(self, state):
        raise NotImplementedError
    

In [22]:
import math

class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def __gr__(self, other):
        return self.path_cost > other.path_cost
        
    def __lt__(self, other):
        return self.path_cost <= other.path_cost

    def __repr__(self):
        if self.action == None:
            sgn = ''
        elif self.state.b:
            sgn = '+'
        else:
            sgn = '-'
        return "<Action from Parent: {}{}, Current Step: {},  depth: {}>".format(sgn,self.action, self.state, self.depth)

    def expand(self, problem):
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        next_state = problem.result(self.state, action)
        next_node = Node(next_state, self, action,
                    problem.path_cost(self.path_cost, self.state,
                                      action, next_state))
        return next_node
    
    def solution(self):
        return [node.action for node in self.path()[1:]]

    def path(self):
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))
    
    def getDepth(self):
        return self.depth

failure = Node(failState, path_cost=math.inf) # Indicates an algorithm couldn't find a solution.

In [23]:
class Cym(Problem):
    initial = Estado(3,3,1)
    goal = Estado(0,0,0)
    
    def __init__(self):
        Problem.__init__(self,self.initial,self.goal)

    def actions(self, estado):
        actions = []
        if estado.b == 1: # es 1, es decir, va a transportar personas del lado inicial al final
            if estado.c >= 2:
                actions = actions + [(1,0),(2,0)]
                if estado.m >= 2:
                    actions = actions + [(0,1),(0,2),(1,1)]
                elif estado.m == 1:
                    actions = actions + [(0,1),(1,1)]
            elif estado.c == 1:
                actions.append((1,0))
                if estado.m >= 2:
                    actions = actions + [(0,1),(0,2),(1,1)]
                elif estado.m == 1:
                    actions = actions + [(0,1),(1,1)]
        else: # es 0, es decir, va a transportar personas del lado final al inicial 
            if 3-estado.c >= 2:
                actions = actions + [(1,0),(2,0)]
                if 3-estado.m >= 2:
                    actions = actions + [(0,1),(0,2),(1,1)]
                elif 3-estado.m == 1:
                    actions = actions + [(0,1),(1,1)]
            elif 3-estado.c == 1:
                actions.append((1,0))
                if 3-estado.m >= 2:
                    actions = actions + [(0,1),(0,2),(1,1)]
                elif 3-estado.m == 1:
                    actions = actions + [(0,1),(1,1)]
        return actions
    
    def result(self, state, action):
        if state.b == 1: # va a restar 
            nuevo = Estado(state.c-action[0], state.m-action[1], (state.b+1)%2)
        else: # va a sumar
            nuevo = Estado(state.c+action[0], state.m+action[1], (state.b+1)%2)
        if nuevo.isValid(): # checar si los canibales no sobrepasan a los misioneros
            return nuevo
        else:
            return failState
        
    def goal_test(self, state):
        return state == self.goal

## best first search


In [24]:
import heapq


def best_first_graph_search(problem, f, display=False):
    f = memoize(f, 'f')
    node = Node(problem.initial)
    frontier = PriorityQueue('min', f)
    frontier.append(node)
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            if display:
                print(len(explored), "paths have been expanded and", len(frontier), "paths remain in the frontier")
            return node
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child)
            elif child in frontier:
                if f(child) < frontier[child]:
                    del frontier[child]
                    frontier.append(child)
    return None



## astar search 


In [25]:
def astar_search(problem, h=None, display=False):
    h = memoize(h or problem.h, 'h')
    return best_first_graph_search(problem, lambda n: n.path_cost + h(n), display) 

In [26]:
problema = Cym()
bfs = astar_search(problema, lambda n: n.path_cost, False)
for node in bfs.path():
    print(node)

print('------------------------------')


<Action from Parent: None, Current Step: (missionaries: 3, cannibals:3) b = 1,  depth: 0>
<Action from Parent: -(1, 1), Current Step: (missionaries: 2, cannibals:2) b = 0,  depth: 1>
<Action from Parent: +(0, 1), Current Step: (missionaries: 2, cannibals:3) b = 1,  depth: 2>
<Action from Parent: -(2, 0), Current Step: (missionaries: 0, cannibals:3) b = 0,  depth: 3>
<Action from Parent: +(1, 0), Current Step: (missionaries: 1, cannibals:3) b = 1,  depth: 4>
<Action from Parent: -(0, 2), Current Step: (missionaries: 1, cannibals:1) b = 0,  depth: 5>
<Action from Parent: +(1, 1), Current Step: (missionaries: 2, cannibals:2) b = 1,  depth: 6>
<Action from Parent: -(0, 2), Current Step: (missionaries: 2, cannibals:0) b = 0,  depth: 7>
<Action from Parent: +(1, 0), Current Step: (missionaries: 3, cannibals:0) b = 1,  depth: 8>
<Action from Parent: -(2, 0), Current Step: (missionaries: 1, cannibals:0) b = 0,  depth: 9>
<Action from Parent: +(0, 1), Current Step: (missionaries: 1, cannibals:1

## Uniform cost search 


In [27]:
def uniform_cost_search(problem, display=False):
    return best_first_graph_search(problem, lambda node: node.path_cost, display)


In [28]:

problema = Cym()
graph = uniform_cost_search(problema, False)
for node in graph.path():
    print(node)

print('------------------------------')


<Action from Parent: None, Current Step: (missionaries: 3, cannibals:3) b = 1,  depth: 0>
<Action from Parent: -(1, 1), Current Step: (missionaries: 2, cannibals:2) b = 0,  depth: 1>
<Action from Parent: +(0, 1), Current Step: (missionaries: 2, cannibals:3) b = 1,  depth: 2>
<Action from Parent: -(2, 0), Current Step: (missionaries: 0, cannibals:3) b = 0,  depth: 3>
<Action from Parent: +(1, 0), Current Step: (missionaries: 1, cannibals:3) b = 1,  depth: 4>
<Action from Parent: -(0, 2), Current Step: (missionaries: 1, cannibals:1) b = 0,  depth: 5>
<Action from Parent: +(1, 1), Current Step: (missionaries: 2, cannibals:2) b = 1,  depth: 6>
<Action from Parent: -(0, 2), Current Step: (missionaries: 2, cannibals:0) b = 0,  depth: 7>
<Action from Parent: +(1, 0), Current Step: (missionaries: 3, cannibals:0) b = 1,  depth: 8>
<Action from Parent: -(2, 0), Current Step: (missionaries: 1, cannibals:0) b = 0,  depth: 9>
<Action from Parent: +(0, 1), Current Step: (missionaries: 1, cannibals:1

## Breadth_first_search

In [29]:
from collections import deque

FIFOQueue = deque

def breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    node = Node(problem.initial)
    if problem.goal_test(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in node.expand(problem):
            s = child.state
            if problem.goal_test(s):
                return child
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure

In [30]:
problema = Cym()
bfs = breadth_first_search(problema)
for node in bfs.path():
    print(node)

<Action from Parent: None, Current Step: (missionaries: 3, cannibals:3) b = 1,  depth: 0>
<Action from Parent: -(2, 0), Current Step: (missionaries: 1, cannibals:3) b = 0,  depth: 1>
<Action from Parent: +(1, 0), Current Step: (missionaries: 2, cannibals:3) b = 1,  depth: 2>
<Action from Parent: -(2, 0), Current Step: (missionaries: 0, cannibals:3) b = 0,  depth: 3>
<Action from Parent: +(1, 0), Current Step: (missionaries: 1, cannibals:3) b = 1,  depth: 4>
<Action from Parent: -(0, 2), Current Step: (missionaries: 1, cannibals:1) b = 0,  depth: 5>
<Action from Parent: +(1, 1), Current Step: (missionaries: 2, cannibals:2) b = 1,  depth: 6>
<Action from Parent: -(0, 2), Current Step: (missionaries: 2, cannibals:0) b = 0,  depth: 7>
<Action from Parent: +(1, 0), Current Step: (missionaries: 3, cannibals:0) b = 1,  depth: 8>
<Action from Parent: -(2, 0), Current Step: (missionaries: 1, cannibals:0) b = 0,  depth: 9>
<Action from Parent: +(1, 0), Current Step: (missionaries: 2, cannibals:0

## Depth first tree search


In [31]:
def depth_first_tree_search(problem):
    node = Node(problem.initial)
    if problem.goal_test(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in node.expand(problem):
            s = child.state
            if problem.goal_test(s):
                return child
            if s not in reached:
                reached.add(s)
                frontier.append(child)
    return failure

In [32]:
problema = Cym()
bfs = depth_first_tree_search(problema)
for node in bfs.path():
    print(node)

<Action from Parent: None, Current Step: (missionaries: 3, cannibals:3) b = 1,  depth: 0>
<Action from Parent: -(1, 1), Current Step: (missionaries: 2, cannibals:2) b = 0,  depth: 1>
<Action from Parent: +(0, 1), Current Step: (missionaries: 2, cannibals:3) b = 1,  depth: 2>
<Action from Parent: -(2, 0), Current Step: (missionaries: 0, cannibals:3) b = 0,  depth: 3>
<Action from Parent: +(1, 0), Current Step: (missionaries: 1, cannibals:3) b = 1,  depth: 4>
<Action from Parent: -(0, 2), Current Step: (missionaries: 1, cannibals:1) b = 0,  depth: 5>
<Action from Parent: +(1, 1), Current Step: (missionaries: 2, cannibals:2) b = 1,  depth: 6>
<Action from Parent: -(0, 2), Current Step: (missionaries: 2, cannibals:0) b = 0,  depth: 7>
<Action from Parent: +(1, 0), Current Step: (missionaries: 3, cannibals:0) b = 1,  depth: 8>
<Action from Parent: -(2, 0), Current Step: (missionaries: 1, cannibals:0) b = 0,  depth: 9>
<Action from Parent: +(0, 1), Current Step: (missionaries: 1, cannibals:1

## Depth first graph search

Es la misma implementación que depth tree, pero sí checa cuando dos paths llegan al estado y elige el primero que encuentra

In [33]:
def depth_first_graph_search(problem):
    frontier = [(Node(problem.initial))]
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        explored.add(node.state)
        frontier.extend(child for child in node.expand(problem)
                        if child.state not in explored and child not in frontier)
    return None


In [34]:
problema = Cym()
bfs = depth_first_graph_search(problema)
for node in bfs.path():
    print(node)

<Action from Parent: None, Current Step: (missionaries: 3, cannibals:3) b = 1,  depth: 0>
<Action from Parent: -(1, 1), Current Step: (missionaries: 2, cannibals:2) b = 0,  depth: 1>
<Action from Parent: +(0, 1), Current Step: (missionaries: 2, cannibals:3) b = 1,  depth: 2>
<Action from Parent: -(2, 0), Current Step: (missionaries: 0, cannibals:3) b = 0,  depth: 3>
<Action from Parent: +(1, 0), Current Step: (missionaries: 1, cannibals:3) b = 1,  depth: 4>
<Action from Parent: -(0, 2), Current Step: (missionaries: 1, cannibals:1) b = 0,  depth: 5>
<Action from Parent: +(1, 1), Current Step: (missionaries: 2, cannibals:2) b = 1,  depth: 6>
<Action from Parent: -(0, 2), Current Step: (missionaries: 2, cannibals:0) b = 0,  depth: 7>
<Action from Parent: +(1, 0), Current Step: (missionaries: 3, cannibals:0) b = 1,  depth: 8>
<Action from Parent: -(2, 0), Current Step: (missionaries: 1, cannibals:0) b = 0,  depth: 9>
<Action from Parent: +(0, 1), Current Step: (missionaries: 1, cannibals:1

## Iterative deepening search


In [35]:
def iterative_deepening_search(problem):
    for depth in range(20):
        result = depth_limited_search(problem, depth)
        if result != 'cutoff':
            return depth_limited_search(problem, depth)
        
def depth_limited_search(problem, limit=10):
    return recursive_dls(Node(problem.initial), problem, limit)

def recursive_dls(node, problem, limit):
    if problem.goal_test(node.state):
        return node
    elif limit == 0:
        return 'cutoff'
    else:
        cutoff_occurred = False
        for child in node.expand(problem):
            result = recursive_dls(child, problem, limit - 1)
            if result == 'cutoff':
                cutoff_occurred = True
            elif result is not None:
                return result
        return 'cutoff' if cutoff_occurred else None

In [36]:
problema = Cym()
bfs = iterative_deepening_search(problema)
for node in bfs.path():
    print(node)

<Action from Parent: None, Current Step: (missionaries: 3, cannibals:3) b = 1,  depth: 0>
<Action from Parent: -(2, 0), Current Step: (missionaries: 1, cannibals:3) b = 0,  depth: 1>
<Action from Parent: +(1, 0), Current Step: (missionaries: 2, cannibals:3) b = 1,  depth: 2>
<Action from Parent: -(2, 0), Current Step: (missionaries: 0, cannibals:3) b = 0,  depth: 3>
<Action from Parent: +(1, 0), Current Step: (missionaries: 1, cannibals:3) b = 1,  depth: 4>
<Action from Parent: -(0, 2), Current Step: (missionaries: 1, cannibals:1) b = 0,  depth: 5>
<Action from Parent: +(1, 1), Current Step: (missionaries: 2, cannibals:2) b = 1,  depth: 6>
<Action from Parent: -(0, 2), Current Step: (missionaries: 2, cannibals:0) b = 0,  depth: 7>
<Action from Parent: +(1, 0), Current Step: (missionaries: 3, cannibals:0) b = 1,  depth: 8>
<Action from Parent: -(2, 0), Current Step: (missionaries: 1, cannibals:0) b = 0,  depth: 9>
<Action from Parent: +(1, 0), Current Step: (missionaries: 2, cannibals:0

Vemos que depende mucho del algoritmo usado, pero todos tienen una profundidad de 11, es decir, que toman 11 pasos diferentes para cruzar a todos 